# DESAFIO

###### Como utilizar a discretização e a classificação da mesma forma em uma empresa ou indústria? Monte um exemplo utilizando o conhecimento abordado no estudo de caso proposto, lembrando que o objetivo é reutilizar o modelo de conhecimento adquirido.
###### 


#### Caso de uso
#### 
###### Previsão de churn

## Prevenção de Abandono de Carrinho em E-commerce

Contexto:

Em uma situação onde existe uma loja online que vende smartphones e quer prever quais clientes estão propensos a abandonar o carrinho de compras antes de concluir a compra. 

O que será feito:
Usaremos técnicas de discretização para categorizar os dados dos clientes e algoritmos de classificação para prever o comportamento de compra.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

#### Simulação dos dados

Temos um conjunto de dados simulados contendo as seguintes informações: tempo gasto na página do produto, número de visitas ao site, quantidade de produtos no carrinho, histórico de compras, e número de abandonos anteriores.

In [3]:
np.random.seed(42)
data = {
    'tempo_na_pagina': np.random.uniform(0.5, 10, 1000),  # tempo em minutos
    'numero_visitas': np.random.randint(1, 20, 1000),
    'qtd_produtos_carrinho': np.random.randint(1, 5, 1000),
    'historico_compras': np.random.randint(0, 3, 1000),
    'abandonos_previos': np.random.randint(0, 5, 1000)
}
df = pd.DataFrame(data)
df

,tempo_na_pagina,numero_visitas,qtd_produtos_carrinho,historico_compras,abandonos_previos
0,4.058131,15,2,0,2
1,9.531786,12,1,2,2
2,7.453942,16,4,1,3
3,6.187256,19,4,2,2
4,1.982177,8,3,1,2
...,...,...,...,...,...
995,1.370030,4,3,0,0
996,9.214479,14,4,0,3
997,1.799777,9,2,2,0
998,9.527255,9,1,1,4


##### Discretização dos Dados
####  

Definindo se o tempo da página foi considerado curto, médio ou longo, com base no tempo. Auxiliando na etapa da análise.

- De 0 a 1 minuto é considerado "Curto".
- De 1 a 3 minutos é considerado "Médio".
- Mais de 3 minutos é considerado "Longo".

In [4]:
bins_tempo = [0, 1, 3, np.inf]
labels_tempo = ['Curto', 'Médio', 'Longo']
df['tempo_na_pagina_cat'] = pd.cut(df['tempo_na_pagina'], bins=bins_tempo, labels=labels_tempo)
df

,tempo_na_pagina,numero_visitas,qtd_produtos_carrinho,historico_compras,abandonos_previos,tempo_na_pagina_cat
0,4.058131,15,2,0,2,Longo
1,9.531786,12,1,2,2,Longo
2,7.453942,16,4,1,3,Longo
3,6.187256,19,4,2,2,Longo
4,1.982177,8,3,1,2,Médio
...,...,...,...,...,...,...
995,1.370030,4,3,0,0,Médio
996,9.214479,14,4,0,3,Longo
997,1.799777,9,2,2,0,Médio
998,9.527255,9,1,1,4,Longo


In [6]:

bins_visitas = [0, 5, 15, np.inf]
labels_visitas = ['Baixo', 'Médio', 'Alto']
df['numero_visitas_cat'] = pd.cut(df['numero_visitas'], bins=bins_visitas, labels=labels_visitas)
df

,tempo_na_pagina,numero_visitas,qtd_produtos_carrinho,historico_compras,abandonos_previos,tempo_na_pagina_cat,numero_visitas_cat
0,4.058131,15,2,0,2,Longo,Médio
1,9.531786,12,1,2,2,Longo,Médio
2,7.453942,16,4,1,3,Longo,Alto
3,6.187256,19,4,2,2,Longo,Alto
4,1.982177,8,3,1,2,Médio,Médio
...,...,...,...,...,...,...,...
995,1.370030,4,3,0,0,Médio,Baixo
996,9.214479,14,4,0,3,Longo,Médio
997,1.799777,9,2,2,0,Médio,Médio
998,9.527255,9,1,1,4,Longo,Médio


#### Contagem de churn (evasão prévia)

Churn é um conceito fundamental em retenção de clientes e é usado para medir a perda de clientes. Ao prever o churn, as empresas podem agir proativamente para reduzir a perda de clientes, aumentando a retenção e, consequentemente, a receita e a sustentabilidade do negócio.

In [8]:
df['churn'] = np.where(df['abandonos_previos'] > 2, 1, 0)
df

,tempo_na_pagina,numero_visitas,qtd_produtos_carrinho,historico_compras,abandonos_previos,tempo_na_pagina_cat,numero_visitas_cat,churn
0,4.058131,15,2,0,2,Longo,Médio,0
1,9.531786,12,1,2,2,Longo,Médio,0
2,7.453942,16,4,1,3,Longo,Alto,1
3,6.187256,19,4,2,2,Longo,Alto,0
4,1.982177,8,3,1,2,Médio,Médio,0
...,...,...,...,...,...,...,...,...
995,1.370030,4,3,0,0,Médio,Baixo,0
996,9.214479,14,4,0,3,Longo,Médio,1
997,1.799777,9,2,2,0,Médio,Médio,0
998,9.527255,9,1,1,4,Longo,Médio,1


#### Seleção das features

- Essa função é usada para converter variáveis categóricas em variáveis dummy ou variáveis indicadoras.
- Em vez de manter uma variável categórica com valores como "Curto", "Médio", "Longo", get_dummies cria uma nova coluna para cada categoria, onde os valores são 0 ou 1. Por exemplo, a coluna "tempo_na_pagina_cat" que tem três categorias ("Curto", "Médio", "Longo") se torna três colunas: "tempo_na_pagina_cat_Médio", "tempo_na_pagina_cat_Longo", e "tempo_na_pagina_cat_Curto". Cada uma dessas colunas terá 1 ou 0, indicando se essa categoria é aplicável a uma linha específica.

In [15]:
X = pd.get_dummies(df[['tempo_na_pagina_cat', 'numero_visitas_cat', 'qtd_produtos_carrinho', 'historico_compras']], drop_first=True)
y = df['churn']

#### Divisão dos dados em treinamento e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


#### Inicialização dos modelos

In [20]:
models = {
    "Árvore de Decisão": DecisionTreeClassifier(random_state=42),
    "Rede Neural MLP": MLPClassifier(max_iter=300, random_state=42),
    "Naive Bayes": GaussianNB()
}

# Treinamento e Avaliação dos Modelos
results = []

for model_name, model in models.items():
    # Treinamento do modelo
    model.fit(X_train, y_train)
    
    # Previsão
    y_pred = model.predict(X_test)
    
    # Acurácia
    accuracy = accuracy_score(y_test, y_pred)
    
    # Validação Cruzada 10-fold
    cross_val_accuracy = cross_val_score(model, X, y, cv=10).mean()
    
    # Coleta de resultados
    results.append({
        "Modelo": model_name,
        "Acurácia (%)": round(accuracy * 100, 2),
        "Validação Cruzada (10-fold)": round(cross_val_accuracy * 100, 2)
    })

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


#### Data frame que irá exibir os resultados

In [21]:
# Criar um DataFrame para exibir os resultados
df_results = pd.DataFrame(results)
df_results

,Modelo,Acurácia (%),Validação Cruzada (10-fold)
0,Árvore de Decisão,53.33,58.8
1,Rede Neural MLP,53.33,59.0
2,Naive Bayes,60.00,60.7


#### Resultados

**Árvore de Decisão:**

- Acurácia: 53.33% — Isso significa que o modelo previu corretamente cerca de 53.33% das vezes no conjunto de teste.
- Validação Cruzada (10-fold): 58.8% — A validação cruzada sugere que, em média, o modelo teve um desempenho de 58.8% de acurácia em diferentes subconjuntos dos dados.

Interpretação: A Árvore de Decisão teve um desempenho modesto, sugerindo que o modelo pode estar tendo dificuldades para capturar os padrões complexos nos dados.

**Rede Neural MLP:**

- Acurácia: 53.33% — Igual à Árvore de Decisão, indicando que, no conjunto de teste, a Rede Neural MLP também não conseguiu superar este limite.
- Validação Cruzada (10-fold): 59.0% — A validação cruzada sugere uma ligeira melhora em relação à Árvore de Decisão, mas ainda é baixa.

Interpretação: Embora a Rede Neural MLP tenha um potencial para capturar padrões complexos, esses resultados indicam os dados disponíveis não são ideais para esse tipo de modelo.

**Naive Bayes:**

- Acurácia: 60.00% — O Naive Bayes teve o melhor desempenho em termos de acurácia, prevendo corretamente 60% dos casos no conjunto de teste.
- Validação Cruzada (10-fold): 60.7% — Isso indica uma consistência decente na generalização dos dados e sugere que o Naive Bayes está se saindo melhor que os outros modelos nesta tarefa específica.

Interpretação: O Naive Bayes, sendo um modelo mais simples e probabilístico, conseguiu captar padrões nos dados de maneira mais eficaz que os outros dois modelos. Isso pode indicar que os dados são adequados para um modelo mais linear ou que as suposições simplificadas do Naive Bayes funcionam bem com esse conjunto de dados.